<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/ejemyr/Lab-1/ejemyr_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: Matrix algorithms**
**Christoffer Ejemyr**

# **Abstract**

In this report we implement the most fundamental vector and matrix algorithms. They are compared to the solutions of the highly regarded package numpy. All algorithms passed down to the seventh decimal.

# **About the code**

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Christoffer Ejemyr (ejemyr@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [2]:
# Load neccessary modules.
# from google.colab import files

import time
import numpy as np
import unittest

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

class Tests(unittest.TestCase):
    @staticmethod
    def check_accuracy(est, true, decimal = 7):
        np.testing.assert_almost_equal(est, true, decimal=decimal)

    @staticmethod
    def check_accuracy_multiple_random(num_of_tests, generating_func, decimal = 7):
        for i in range(num_of_tests):
            est, true = generating_func()
            Tests.check_accuracy(est, true, decimal)


# **Introduction**

In this lab we will define multiplication of the most fundamental parts of linear algebra; vectors and matrices. They will ble implemented acording to the algorithms described in the course's literature.

# **Methods**

### Scalar product

In [3]:
def scalar_product(x, y):
    if type(x) != np.ndarray:
        try:
            x = np.array(x)
        except:
            raise Exception("Vector format error.\n" + "x: " + str(x) + "\ny: " + str(y))
    if type(y) != np.ndarray:
        try:
            y = np.array(y)
        except:
            raise Exception("Vector format error.\n" + "x: " + str(x) + "\ny: " + str(y))

    if x.ndim != 1 or y.ndim != 1:
        raise Exception("Vector dimentions error.\n" + "x: " + str(x) + "\ny: " + str(y))
    if x.size != y.size:
        raise Exception("Vector formats don't agree.\n" + "x: " + str(x) + "\ny: " + str(y))
    
    sum = 0
    for i in range(len(x)):
        sum += x[i] * y[i]

    return sum

In [4]:
class Tests(Tests):
    def test_scalar_product(self):
        with self.assertRaises(Exception):
            scalar_product([1,2], [1,2,3])
        with self.assertRaises(Exception):
            scalar_product([[0,0],[0,0]], [[0,0],[0,0]])
        with self.assertRaises(Exception):
            scalar_product(["s",2], [1,3])
        with self.assertRaises(Exception):
            scalar_product("Hej", [1,2,3])

        min_length = 1
        max_length = 100
        def genetator():
            n = np.random.randint(min_length, max_length)
            a = np.random.rand(n)
            b = np.random.rand(n)
            return scalar_product(a, b), a.dot(b)

        Tests.check_accuracy_multiple_random(1000, genetator)

### Matrix-vector product

In [5]:
def matrix_vector_product(M, x):
    if type(M) != np.ndarray:
        try:
            M = np.array(M)
        except:
            raise Exception("Matrix format error.\n" + "M: " + str(M))
    if type(x) != np.ndarray:
        try:
            x = np.array(x)
        except:
            raise Exception("Vector format error.\n" + "x: " + str(x))

    if x.ndim != 1 or M.ndim != 2:
        raise Exception("Matrix or vector dimentions error.\n" + "M: " + str(M) + "\nx: " + str(x))
    if M.shape[1] != x.size:
        raise Exception("Matrix and vector formats don't agree.\n" + "M: " + str(M) + "\nx: " + str(x))

    b = np.zeros(M.shape[0])
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            b[i] += M[i, j] * x[j]

    return b

In [6]:
class Tests(Tests):
    def test_matrix_vector_product(self):
        with self.assertRaises(Exception):
            scalar_product([[0,0],[0,0]], [1,2,3])
        with self.assertRaises(Exception):
            scalar_product([[0,0],[0,0]], ["Hej", "Hå"])

        min_length = 1
        max_length = 100
        def genetator():
            n = np.random.randint(min_length, max_length)
            m = np.random.randint(min_length, max_length)
            x = np.random.rand(n)
            M = np.random.rand(m, n)
            return matrix_vector_product(M, x), M.dot(x)

        Tests.check_accuracy_multiple_random(1000, genetator)

### Matrix-Matrix product

In [7]:
def matrix_matrix_product(A, B):
    def format_test(M, name):
        if type(M) != np.ndarray:
            try:
                M = np.array(M)
            except:
                raise Exception("Matrix format error.\n" + name + ": " + str(M))

        if M.ndim != 2:
            raise Exception("Matrix dimentions error.\n" + name + ": " + str(M))
    format_test(A, "A")
    format_test(B, "B")
    
    if A.shape[1] != B.shape[0]:
        raise Exception("Matrix formats don't agree.\n" + "A: " + str(A) + "\nB: " + str(B))
    
    
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]

    return C

In [8]:
class Tests(Tests):
    def test_matrix_matrix_product(self):
        with self.assertRaises(Exception):
            scalar_product([[0,0,0],[0,0,0]], [[0,0],[0,0]])
        with self.assertRaises(Exception):
            scalar_product([[0,0],[0,0]], [["Hej"], ["Hå"]])

        min_length = 1
        max_length = 100
        def genetator():
            i = np.random.randint(min_length, max_length)
            j = np.random.randint(min_length, max_length)
            k = np.random.randint(min_length, max_length)
            A = np.random.rand(i, j)
            B = np.random.rand(j, k)
            return matrix_matrix_product(A, B), A.dot(B)

        Tests.check_accuracy_multiple_random(100, genetator)

# **Results**

To test the methods and to receive a result we test both the error handling and the accuracy of the methods. Sice we don't have any known solution to the expresions tested we compare with a solution known to be exact enough. It is difficult to know where the limit of an "accurate" result lie. In this lab i've chosen to check that all result lie within a 7 decimal margin.

The test first checks that exceptions are raised when entering incompatible inputs, and then somwhere between 100 and 1000 random vectors or matricies are tested for accuracy according to the method described above.

In [9]:
suite = unittest.TestSuite()
suite.addTest(Tests('test_scalar_product'))
suite.addTest(Tests('test_matrix_vector_product'))
suite.addTest(Tests('test_matrix_matrix_product'))

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 20.209s

OK


All the test passes, meaning that the potential floating-point-errors is within an acceptable level.

# **Discussion**

Rather non-supricing all methods passed the tests. Since all implemented algorithms have a unique and mathematically explicit way of calculationg the result there ain't much room for error.